# Plane Ticket Price & Recommendation Tool

I wanted to spice things up to show my understanding of the use of AI models from the course, so I built a small recommendation tool.

**Flight data** from the [Amadeus Open API](https://developers.amadeus.com/) (free tier).

This tool recommends plane ticket prices and airlines based on:
- **User preferences** (most affordable, value for money, or best airline)
- **Current location and destination**
- **Duration of travel**
- **Number of adults and children travelling**

## Week 1 - Day One Challenge

In [ ]:
# Imports
import os
import json
from datetime import datetime
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

AMADEUS_AUTH_URL = "https://test.api.amadeus.com/v1/security/oauth2/token"
AMADEUS_FLIGHTS_URL = "https://test.api.amadeus.com/v2/shopping/flight-offers"

In [ ]:
# Environment Setup
load_dotenv(override=True)
openrouter_key = os.getenv("OPENROUTER_API_KEY")
amadeus_id = os.getenv("AMADEUS_CLIENT_ID")
amadeus_secret = os.getenv("AMADEUS_CLIENT_SECRET")

print("OpenRouter:", "OK" if openrouter_key else "missing")
print("Amadeus:", "OK" if (amadeus_id and amadeus_secret) else "mock")


In [ ]:
# Amadeus API - direct HTTP (no SDK)
def get_amadeus_token():
    r = requests.post(AMADEUS_AUTH_URL, data={"grant_type": "client_credentials", "client_id": amadeus_id, "client_secret": amadeus_secret},
                      headers={"Content-Type": "application/x-www-form-urlencoded"})
    return r.json().get("access_token") if r.status_code == 200 else None

def amadeus_fetch_flights(origin, destination, departure_date, return_date, adults=1, children=0):
    token = get_amadeus_token()
    if not token:
        return None
    params = {"originLocationCode": origin, "destinationLocationCode": destination, "departureDate": departure_date,
              "returnDate": return_date, "adults": adults}
    if children:
        params["children"] = children
    r = requests.get(AMADEUS_FLIGHTS_URL, params=params, headers={"Authorization": f"Bearer {token}"})
    return (r.json().get("data") or [])[:7] if r.status_code == 200 else None


In [ ]:
# Test Amadeus API
def test_amadeus_api():
    if not amadeus_id or not amadeus_secret:
        print("Missing Amadeus credentials")
        return False
    offers = amadeus_fetch_flights("JFK", "LHR", "2026-06-01", "2026-06-08", adults=1)
    ok = offers is not None
    print(f"Amadeus: {'OK' if ok else 'error'} - {len(offers or [])} offers")
    return ok

test_amadeus_api()


In [ ]:
# TicketRecommender
class TicketRecommender:
    SYSTEM_PROMPT = """You are a travel expert. Recommend top 3 tickets ranked 1-3. 
    Consider price, airline rating, value. Use full location names, show duration, adults/children. 
    Include booking links as [Book here](url). Markdown. 
    Add some explanation for your reason for each recommendation"""

    def __init__(self, model="openai/gpt-4o-mini"):
        self.model = model
        self.client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=os.getenv("OPENROUTER_API_KEY"))

    def recommend(self, origin, destination, start_date, return_date, adults=1, children=0, preference="value_for_money"):
        duration_days = (datetime.strptime(return_date, "%Y-%m-%d") - datetime.strptime(start_date, "%Y-%m-%d")).days
        input_data = {"origin": origin, "destination": destination, "start_date": start_date, "return_date": return_date,
                      "duration_days": duration_days, "adults": adults, "children": children}
        flight_data = self._fetch_flight_data(origin, destination, start_date, return_date, adults, children)
        if isinstance(flight_data, dict) and "error" in flight_data:
            return flight_data["error"]
        return self._get_ai_recommendation(input_data, flight_data, preference)

    def _fetch_flight_data(self, origin, destination, start_date, return_date, adults=1, children=0):
        if amadeus_id and amadeus_secret:
            offers = amadeus_fetch_flights(origin, destination, start_date, return_date, adults, children)
            if offers is not None:
                return offers if offers else self._mock_flight_data()
            return {"error": "Travel API error"}
        return self._mock_flight_data()

    def _mock_flight_data(self):
        return {"flights": [{"airline": "British Airways", "price": 450, "rating": 4.2, "stops": 0, "link": "https://www.britishairways.com/travel/book/public/en_us"},
                {"airline": "United Airlines", "price": 380, "rating": 3.8, "stops": 1, "link": "https://www.united.com/en/us/book-flight"},
                {"airline": "Delta", "price": 520, "rating": 4.5, "stops": 0, "link": "https://www.delta.com/flight-search"}]}

    def _get_ai_recommendation(self, input_data, api_response, preference):
        prompt = f"User: {json.dumps(input_data)}\nFlights: {json.dumps(api_response)}\nPreference: {preference}\nTop 3 recommendations."
        try:
            r = self.client.chat.completions.create(model=self.model, messages=[{"role": "system", "content": self.SYSTEM_PROMPT}, {"role": "user", "content": prompt}])
            return r.choices[0].message.content
        except Exception as e:
            return f"AI error: {e}"


In [ ]:
# Demo: Get ticket recommendation
recommender = TicketRecommender()
result = recommender.recommend(
    origin="LAS",
    destination="LHR",
    start_date="2026-04-01",
    return_date="2026-04-08",
    adults=2,
    children=1,
    preference="value_for_money"
)
display(Markdown(result))